## DEMO AND TUTORIAL FOR LES FURNITURES

### Setting directory

In [ ]:
import os
dirmain = os.getcwd()
print(dirmain)

### Cloning repo

In [ ]:
!git clone https://github.com/AryamanSharma17/Grounded-Segment-Anything

### Installing Meta files

In [ ]:
# dirmain = dir + '/Grounded-Segment-Anything'
os.chdir(dirmain + '/Grounded-Segment-Anything')
%pip install -q -r requirements.txt
os.chdir(dirmain + '/Grounded-Segment-Anything/segment_anything')
%pip install -q .
os.chdir(dirmain + '/Grounded-Segment-Anything/GroundingDINO')
!pip install -q .
os.chdir(dirmain + '/Grounded-Segment-Anything')


New directory should be: Main Directory + Grounded-Segment-Anything

In [ ]:
os.getcwd()

In [ ]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

import argparse
import copy

from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from torchvision.ops import box_convert

# Grounding DINO
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
from segment_anything import build_sam, SamPredictor 
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import csv
import pandas as pd


from huggingface_hub import hf_hub_download

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file) 
    args.device = device
    model = build_model(args)
    
    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location=device)
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model   

Grounding DINO

In [ ]:
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"


groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device='cuda')

SAM

In [ ]:
if os.path.exists(dirmain+'/Grounded-Segment-Anything/sam_vit_b_01ec64.pth')==False:
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
sam_checkpoint = 'sam_vit_b_01ec64.pth'
sam_predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))


LOAD IMAGE

In [ ]:
test_image = dirmain +'/Resource/classroom_simple.jpg'
image_source, image = load_image(test_image)
source_image = Image.fromarray(image_source)

In [ ]:
source_image = source_image.resize((500,500))
image_source = np.asarray(source_image)
Image.fromarray(image_source)

In [ ]:
image_source.shape

Bounding box and center generation with GDINO

In [ ]:
def detect(image, text_prompt, model, box_threshold = 0.4, text_threshold = 0.2):
  boxes, logits, phrases = predict(
      model=model, 
      image=image, 
      caption=text_prompt,
      box_threshold=box_threshold,
      text_threshold=text_threshold
  )

  annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
  annotated_frame = annotated_frame[...,::-1] # BGR to RGB 
  return annotated_frame, boxes,phrases, logits


def gen_mask_Gd(image_source, boxes):
    h,w,_ = image_source.shape
    boxes_unnorm = boxes * torch.Tensor([w,h,w,h]) 
    boxes_xyxy = box_convert(boxes =boxes_unnorm, in_fmt = "cxcywh", out_fmt = "xyxy").numpy()
    mask = np.zeros_like(image_source)
    for box in boxes_xyxy:
        x0, y0,x1,y1 = box
        mask[int(y0):int(y1), int(x0):int(x1),:] = 255
    return mask, boxes_xyxy, boxes_unnorm

def center(centerx,centery,detected_boxes):
    for i in range(len(detected_boxes)):
        centerx.append(int(detected_boxes[i][0] + detected_boxes[i][2])/2)
        centery.append(int(detected_boxes[i][1] + detected_boxes[i][3])/2)
    return centerx,centery

In [ ]:
annotated_frame, detected_boxes,phrases, logits = detect(image, text_prompt="chair. table. sofa. door. window", model=groundingdino_model, box_threshold=0.5 )
anoim = Image.fromarray(annotated_frame)

In [ ]:
mask,box_change,box_unnormalized = gen_mask_Gd(image_source,detected_boxes)

In [ ]:
centerx = []
centery = []

centerx,centery = center(centerx,centery,box_change)

In [ ]:
import matplotlib.pyplot as plt
c = np.array(["red","green","yellow","cyan","orange", "lime", "pink","Maroon", "purple", "aquamarine", "olive", "khakhi"])

plt.figure(figsize=(10,10))
scatter = plt.scatter(centerx,centery )
plt.imshow(anoim)
plt.grid()

SAM: Segmentation

In [ ]:
def segment(image, sam_model, boxes):
  sam_model.set_image(image)
  H, W, _ = image.shape
  boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

  transformed_boxes = sam_model.transform.apply_boxes_torch(boxes_xyxy.to(device), image.shape[:2])
  masks, _, _ = sam_model.predict_torch(
      point_coords = None,
      point_labels = None,
      boxes = transformed_boxes,
      multimask_output = False,
      )
  return masks.cuda()
  

def draw_mask(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.cpu().reshape(h, w, 1) * color.reshape(1, 1, -1)
    
    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image.cpu().numpy() * 255).astype(np.uint8)).convert("RGBA")

    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

Single Mask Creation

In [ ]:
segmented_frame_masks = segment(image_source, sam_predictor, boxes=detected_boxes)
annotated_frame_with_mask = draw_mask(segmented_frame_masks[0][0], annotated_frame)


Multiple Mask fusion

In [ ]:
all_frames = annotated_frame
for i in range(len(segmented_frame_masks)):
  all_frames = draw_mask(segmented_frame_masks[i][0], all_frames)
Image.fromarray(all_frames)

In [ ]:
allmask = segmented_frame_masks[0][0].cpu().numpy()
for i in range(1,len(segmented_frame_masks)):
    allmask += segmented_frame_masks[i][0].cpu().numpy() 

In [ ]:
allmask_image = Image.fromarray(allmask)


In [ ]:
display(allmask_image)

Preprocessing the image to invert the axes

In [ ]:
# allmask_image=allmask_image.rotate(180)
y_main = np.where(allmask==1)[0]
x_main = np.where(allmask==1)[1]

In [ ]:
int_all_mask = np.array(allmask_image, dtype=np.uint8)

### EROSION

In [ ]:
kernel = np.ones((4,4),np.uint8)
erosion = cv2.erode(int_all_mask, kernel, iterations=1)

In [ ]:
y_erode = np.where(erosion==1)[0]
x_erode = np.where(erosion==1)[1]


Eroded Image

In [ ]:
plt.scatter(x_erode,y_erode, 8)
plt.gca().invert_xaxis()

Actual Image

In [ ]:
plt.scatter(x_main,y_main, 8)
plt.gca().invert_yaxis()

In [ ]:
depth = np.zeros(len(phrases))

Creating output file for VR

In [ ]:
data = np.empty((len(phrases),9),dtype=object)
data.fill([])

In [ ]:
for i in range(len(phrases)):
    for j in range(4):
        data[i][j] = (box_change[i][j])
    data[i][4] = centerx[i]
    data[i][5] = centery[i]
    data[i][6] = depth[i]
    data[i][7] = (phrases[i])
    data[i][8] = logits[i].item()

Rectangles : ABCD

A = 0, 1

B = 0, 3

C = 2, 3

D = 2, 1

center = 4, 5

depth centner = 6

furniture type = 7

In [ ]:
df = pd.DataFrame(data=data, columns=['Coord 1','Coord 2','Coord 3','Coord 4','Center_X', 'Center_Y', 'Depth', 'Furniture category', 'Confidence'])
df

In [ ]:
os.chdir(dirmain)

In [ ]:
# moving to the end after depth estimation 
# df.to_csv('Vr.csv')

### DEPTH ESTIMATION

# Setup

In [ ]:
#!pip install --upgrade timm

In [ ]:
!git clone https://github.com/isl-org/ZoeDepth.git

In [ ]:
!pip install timm==0.6.7

In [ ]:
cd ZoeDepth

In [ ]:
#!python sanity.py

# Load the model

In [ ]:
import torch
from zoedepth.utils.misc import get_image_from_url, colorize
from PIL import Image
import matplotlib.pyplot as plt


zoe = torch.hub.load(".", "ZoeD_N", source="local", pretrained=True)

In [ ]:
zoe = zoe.to('cuda')

# Prediction

In [ ]:
#img = Image.open("test.png")
img = Image.open(test_image)
img.convert("RGB")

In [ ]:

depth = zoe.infer_pil(img)


colored_depth = colorize(depth)
fig, axs = plt.subplots(1,2, figsize=(15,7))
for ax, im, title in zip(axs, [img, colored_depth], ['Input', 'Predicted Depth']):
  ax.imshow(im)
  ax.axis('off')
  ax.set_title(title)

In [ ]:
os.chdir(dirmain)

In [ ]:
center_x = df["Center_X"]
center_y = df["Center_Y"]
print(len(df))
for i in range(0, len(df)):
    depth_val = depth[int(center_x[i]), int(center_y[i])]
    df.at[i, 'Depth'] = depth_val

print("depth max is:" + str(depth.max()) + ",depth min is:" + str(depth.min()))

In [ ]:
df

In [ ]:
df.to_csv('Vr.csv')